## mnist字体识别
### 第二篇：使用全连接神经网络实现mnist分类

在keras中，常用的有两种构建模型的方式，即：  

- 序贯模型：将网络定义为一个序列化的组成，类似于堆栈或队列，把不同的预定义模型按照线性管道叠加放到一起；
- 函数化模型：使用keras的函数api，可以构建更复杂的模型。  

我们将使用序贯模型来实现mnist数据集的分类。

---
第一步：导入模块；

In [3]:
from keras.models import Sequential 
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.datasets import mnist

Using TensorFlow backend.
/usr/local/python3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


然后定义一些需要使用的数据；可以暂时不用理会这些变量的含义；

In [14]:
NB_EPOCH = 5
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASS = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
RESHAPED_SIZE = 784

In [7]:
# 加载数据并标准化；这里暂时使用keras提供的类库；
(x_train,y_train),(x_test,y_test) = mnist.load_data()

# 这里的RESHAPED_SIZE表示目标大小；即将28*28的图像转化为784的向量；
x_train = x_train.reshape(x_train.shape[0], RESHAPED_SIZE).astype('float32')
x_test = x_test.reshape(x_test.shape[0],RESHAPED_SIZE).astype('float32')

x_train /= 255
x_test /= 255

# 将模型标签转化为one-hot向量；例如3表示为[0.,0.,0.,1.,0.,0.,0.,0.,0.,0.]
y_train = np_utils.to_categorical(y_train, NB_CLASS)
y_test = np_utils.to_categorical(y_test, NB_CLASS)

下一步，定义模型的结构，在这里我们使用一个简单的多层感知机的结构    

In [8]:
model = Sequential()

# 第一层，有128个神经单元；
model.add(Dense(128, input_shape=(RESHAPED_SIZE,)))
model.add(Activation('relu'))

# 第二层，有128个神经单元；
model.add(Dense(128))
model.add(Activation('relu'))

# 输出层，10个神经单元；
model.add(Dense(10))
model.add(Activation('softmax'))

# summary输出网络结构；
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


然后是网络的编译和训练；

In [10]:
# 编译网络，从而交给keras后端执行；
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

complie函数的参数有以下几个需要注意的：

- 优化器， 这是训练模型时用于更新权重的特定算法，如sgd、adma。
- 目标函数， 也就是损失函数；
- 评估指标，用于评估模型，常见的有3种：accuracy（准确率）、precision（查准率）、recall（查全率）。
一旦模型编译好，就可以使用fit函数进行训练了

In [15]:
# 模型的训练和优化；
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, 
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 41us/step - loss: 0.1295 - acc: 0.9639 - val_loss: 0.1422 - val_acc: 0.9593
Epoch 2/5
48000/48000 [==============================] - 2s 40us/step - loss: 0.1270 - acc: 0.9640 - val_loss: 0.1403 - val_acc: 0.9599
Epoch 3/5
48000/48000 [==============================] - 2s 40us/step - loss: 0.1247 - acc: 0.9649 - val_loss: 0.1388 - val_acc: 0.9606
Epoch 4/5
48000/48000 [==============================] - 2s 39us/step - loss: 0.1224 - acc: 0.9652 - val_loss: 0.1372 - val_acc: 0.9605
Epoch 5/5
48000/48000 [==============================] - 2s 39us/step - loss: 0.1202 - acc: 0.9664 - val_loss: 0.1361 - val_acc: 0.9608


本例子中使用的fit的参数含义如下：

- batch_size : 每次训练和梯度更新块的大小；
- epochs: 迭代次数;这里为了显示方便，只训练5代；
- verbose: 表示在训练过程中的输出情况，0表示不显示数据，1表示显示进度条，2表示只显示一个数据;
- validation_split : 验证数据的使用比例；

现在已经训练好了整个模型，在验证集上准确率为96.8%，最后在测试集上测试观察其准确率；

In [16]:
score = model.evaluate(x_test, y_test, verbose=VERBOSE)
print("Test score:", score[0])
print("Test accuracy:", score[1])

10000/10000 [==============================] - 0s 42us/step
Test score: 0.1310454538155347
Test accuracy: 0.96


可以看到我们的模型在测试集上最终结果为96%，然而，我们还有更好的模型和训练方法。

下一篇将介绍以下内容：
- **使用卷积神经网络模型来优化mnist分类模型；**
- **使用mnist图像来进行训练；**
- **模型的保存和使用；**